## ETAPA 4: MODELOS DE MACHINE LEARNING

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV

# modelos lineales
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor 
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV

# modelos de arboles
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
#!pip install xgboost
#pip install pydot
from xgboost import XGBRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

import pickle


### *Preparar Datos*

In [2]:
# Importar el dataset
df = pd.read_csv("../13 - Exports (preprocesamiento)/inmigrantes_merge.csv")

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Year                               9360 non-null   int64  
 1   Nationality code                   9360 non-null   object 
 2   Sex                                9360 non-null   object 
 3   Age group                          9360 non-null   object 
 4   Immigrant count                    9360 non-null   int64  
 5   Unemployment %                     9360 non-null   float64
 6   Political and Violence Percentile  9360 non-null   float64
 7   Probability of dying young         9360 non-null   float64
 8   Rule of Law Percentile             9360 non-null   float64
 9   Salaried workers %                 9360 non-null   float64
 10  GDP_growth                         9360 non-null   float64
 11  Inflation_annual                   9360 non-null   float

,Year,Nationality code,Sex,Age group,Immigrant count,Unemployment %,Political and Violence Percentile,Probability of dying young,Rule of Law Percentile,Salaried workers %,...,Non-state_deaths,Intrastate_deaths,Interstate_deaths,Number of residents,Political regime,Homicide Rate,Number of Turist,Spanish language,Restricciones_pandemia,Año post_pandemia
0,2008,DZA,Both,0 - 14,759,11.33,14.90,3.7,24.52,67.41,...,0,345,0,51922,3,0.95,44400000,0,0,0
1,2008,PER,Males,35 - 44,2938,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
2,2008,PER,Males,45 - 54,1128,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
3,2008,PER,Males,55 - 64,265,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
4,2008,PER,Males,65+,156,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,2022,PAK,Males,55 - 64,330,5.60,6.60,5.8,25.00,42.14,...,0,670,0,68821,6,4.21,59310000,0,0,1
9356,2022,PAK,Females,55 - 64,146,5.60,6.60,5.8,25.00,42.14,...,0,670,0,31675,6,4.21,59310000,0,0,1
9357,2022,PAK,Both,65+,169,5.60,6.60,5.8,25.00,42.14,...,0,670,0,100496,6,4.21,59310000,0,0,1
9358,2022,PAK,Males,65+,99,5.60,6.60,5.8,25.00,42.14,...,0,670,0,68821,6,4.21,59310000,0,0,1


Primero, evaluremos modelos que no aceptan datos nulos y, posteriormente los que sí los aceptan. Luego compararemos las distintas métricas juntándolas en dataframes para un mejor análásis.

### *Modelos Que No Aceptan Datos Nulos*

Antes de proceder, debemos remover a Senegal de la lista de de paises ya que tenemos datos nulos para la variable Tasa de Homidicidios, de manera que no haya conflicto con los modelos que evaluaremos. Además, haremos de la varaible "Year" una variable ordinal y el resto de variables categóricas a variables dummy (los regímenes políticos ya están en formato ordinal).

En el caso de Year, simplemente restaremos 2007 a la columna entera, y para el resto de las variables objeto usaremos la funcion *.get_dummies()*.

In [3]:
# hacer copia del df removiendo Senegal que presenta datos nulos para tasa de homicidios
df_nonull = df[df['Nationality code'] != 'SEN'].copy()

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_nonull['Year'] = df_nonull['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_nonull = pd.get_dummies(df_nonull)

# Convertir las variables dummies booleanas en "int"
col_bool = df_nonull.select_dtypes(include = ['bool']).columns
df_nonull[col_bool] = df_nonull[col_bool].astype(int)

# Verificar cambio
df_nonull.info()
df_nonull

<class 'pandas.core.frame.DataFrame'>
Index: 9000 entries, 0 to 9359
Data columns (total 70 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year                                      9000 non-null   int64  
 1   Immigrant count                           9000 non-null   int64  
 2   Unemployment %                            9000 non-null   float64
 3   Political and Violence Percentile         9000 non-null   float64
 4   Probability of dying young                9000 non-null   float64
 5   Rule of Law Percentile                    9000 non-null   float64
 6   Salaried workers %                        9000 non-null   float64
 7   GDP_growth                                9000 non-null   float64
 8   Inflation_annual                          9000 non-null   float64
 9   Liberal democracy index                   9000 non-null   float64
 10  Health equality                          

,Year,Immigrant count,Unemployment %,Political and Violence Percentile,Probability of dying young,Rule of Law Percentile,Salaried workers %,GDP_growth,Inflation_annual,Liberal democracy index,...,Continent_America,Continent_Asia,Continent_Europe,Sub-region_Africa,Sub-region_Asia,Sub-region_Central America and Caribbean,Sub-region_European Union,Sub-region_North America,Sub-region_Rest of Europe,Sub-region_South America
0,1,759,11.33,14.90,3.7,24.52,67.41,2.40,15.31,0.164,...,0,0,0,1,0,0,0,0,0,0
1,1,2938,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
2,1,1128,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
3,1,265,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
4,1,156,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,15,330,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9356,15,146,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9357,15,169,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9358,15,99,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0


In [116]:
# hacer copia del df removiendo Senegal que presenta datos nulos para tasa de homicidios
df_nonull = df_nogrupos[df_nogrupos['Nationality code'] != 'SEN'].copy()

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_nonull['Year'] = df_nonull['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_nonull = pd.get_dummies(df_nonull)

# Convertir las variables dummies booleanas en "int"
col_bool = df_nonull.select_dtypes(include = ['bool']).columns
df_nonull[col_bool] = df_nonull[col_bool].astype(int)

# Verificar cambio
df_nonull.info()
df_nonull

<class 'pandas.core.frame.DataFrame'>
Index: 5250 entries, 1 to 9359
Data columns (total 62 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year                                      5250 non-null   int64  
 1   Immigrant count                           5250 non-null   int64  
 2   Liberal democracy index                   5250 non-null   float64
 3   Health equality                           5250 non-null   float64
 4   Judicial accountability                   5250 non-null   float64
 5   Public sector corrupt exchanges           5250 non-null   float64
 6   One-sided violence_deaths                 5250 non-null   int64  
 7   Non-state_deaths                          5250 non-null   int64  
 8   Intrastate_deaths                         5250 non-null   int64  
 9   Interstate_deaths                         5250 non-null   int64  
 10  Number of residents                      

,Year,Immigrant count,Liberal democracy index,Health equality,Judicial accountability,Public sector corrupt exchanges,One-sided violence_deaths,Non-state_deaths,Intrastate_deaths,Interstate_deaths,...,Continent_America,Continent_Asia,Continent_Europe,Sub-region_Africa,Sub-region_Asia,Sub-region_Central America and Caribbean,Sub-region_European Union,Sub-region_North America,Sub-region_Rest of Europe,Sub-region_South America
1,1,2938,0.649,0.40,0.44,0.30,0,0,40,0,...,1,0,0,0,0,0,0,0,0,1
2,1,1128,0.649,0.40,0.44,0.30,0,0,40,0,...,1,0,0,0,0,0,0,0,0,1
3,1,265,0.649,0.40,0.44,0.30,0,0,40,0,...,1,0,0,0,0,0,0,0,0,1
4,1,156,0.649,0.40,0.44,0.30,0,0,40,0,...,1,0,0,0,0,0,0,0,0,1
8,1,4703,0.649,0.40,0.44,0.30,0,0,40,0,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353,15,452,0.234,0.27,0.22,0.23,68,0,670,0,...,0,1,0,0,1,0,0,0,0,0
9355,15,330,0.234,0.27,0.22,0.23,68,0,670,0,...,0,1,0,0,1,0,0,0,0,0
9356,15,146,0.234,0.27,0.22,0.23,68,0,670,0,...,0,1,0,0,1,0,0,0,0,0
9358,15,99,0.234,0.27,0.22,0.23,68,0,670,0,...,0,1,0,0,1,0,0,0,0,0


Separemos el conjunto train/test y escalemos los datos.

In [4]:
# Separar variables input y variable target "Immigrant count" de df_null (dataframe sin atos nulos)
X = df_nonull.drop("Immigrant count", axis = 1) # variables predictoras
y = df_nonull["Immigrant count"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 58) # separar datos en conjunto train y test en un 75% / 25%
scaler = MinMaxScaler() # definir scaler de datos 
X_train = scaler.fit_transform(X_train) # escalar los datos de entrenamiento
X_test = scaler.fit_transform(X_test) # # escalar los datos de prueba

#### Regresion Lineal

In [5]:
model_lineal = LinearRegression() # definicion del modelo 

model_lineal.fit(X_train, y_train) # ajuste del modelo 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_lineal = model_lineal.predict(X_train)
y_test_pred_lineal = model_lineal.predict(X_test)

# Calcular métricas en conjunto train
r2_train_lineal = np.round(r2_score(y_train, y_train_pred_lineal), 3)
rmse_train_lineal = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_lineal)), 0)
mae_train_lineal = np.round(mean_absolute_error(y_train, y_train_pred_lineal), 0)
mape_train_lineal = np.round(mean_absolute_percentage_error(y_train, y_train_pred_lineal), 2)

# Calcular métricas en conjunto test
r2_test_lineal = np.round(r2_score(y_test, y_test_pred_lineal), 3)
rmse_test_lineal = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_lineal)), 0)
mae_test_lineal = np.round(mean_absolute_error(y_test, y_test_pred_lineal), 0)
mape_test_lineal = np.round(mean_absolute_percentage_error(y_test, y_test_pred_lineal), 2)

# Mostrar métricas
print("R2 train:", r2_train_lineal)
print("RMSE - train:", rmse_train_lineal)
print("MAE - train:", mae_train_lineal)
print("MAPE - train:", mape_train_lineal)
print("")
print("R2 test:", r2_test_lineal)
print("RMSE - test:", rmse_test_lineal)
print("MAE - test:", mae_test_lineal)
print("MAPE - test:", mape_test_lineal)

R2 train: 0.445
RMSE - train: 4128.0
MAE - train: 1773.0
MAPE - train: 6.5

R2 test: 0.507
RMSE - test: 3853.0
MAE - test: 1818.0
MAPE - test: 4.65


In [156]:
# Observar Coeficientes de cada variable para cada modelo en un dataframe
coefficients_lineal = pd.DataFrame({'Variable':df_nonull.drop(["Immigrant count"], axis=1, inplace=False).columns})
coefficients_lineal['modelo_lineal']= model_lineal.coef_

# Mostrar coeficientes
coefficients_lineal.head(20)

,Variable,modelo_lineal
0,Year,-3.551678e+03
1,Unemployment %,4.168233e+03
2,Political and Violence Percentile,3.093668e+03
3,Probability of dying young,9.362225e+03
4,Rule of Law Percentile,-1.040345e+04
5,Salaried workers %,-2.363380e+03
6,GDP_growth,-1.647286e+03
7,Inflation_annual,1.010012e+03
8,Liberal democracy index,3.987376e+03
9,Health equality,5.508632e+02


#### Regresion lineal - Huber (ventaja: bajo efecto de outliers)

In [6]:
model_huber = HuberRegressor(epsilon=1.15, alpha = 0.05) # definicion del modelo 

model_huber.fit(X_train, y_train) # ajuste del modelo 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_huber = model_huber.predict(X_train)
y_test_pred_huber = model_huber.predict(X_test)

# Calcular métricas en conjunto train
r2_train_huber = np.round(r2_score(y_train, y_train_pred_huber), 3)
rmse_train_huber = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_huber)), 0)
mae_train_huber = np.round(mean_absolute_error(y_train, y_train_pred_huber), 0)
mape_train_huber = np.round(mean_absolute_percentage_error(y_train, y_train_pred_huber), 2)

# Calcular métricas en conjunto test
r2_test_huber = np.round(r2_score(y_test, y_test_pred_huber), 3)
rmse_test_huber = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_huber)), 0)
mae_test_huber = np.round(mean_absolute_error(y_test, y_test_pred_huber), 0)
mape_test_huber = np.round(mean_absolute_percentage_error(y_test, y_test_pred_huber), 2)

# Mostrar métricas
print("R2 train:", r2_train_huber)
print("RMSE - train:", rmse_train_huber)
print("MAE - train:", mae_train_huber)
print("MAPE - train:", mape_train_huber)
print("")
print("R2 test:", r2_test_huber)
print("RMSE - test:", rmse_test_huber)
print("MAE - test:", mae_test_huber)
print("MAPE - test:", mape_test_huber)

R2 train: 0.249
RMSE - train: 4800.0
MAE - train: 1306.0
MAPE - train: 1.39

R2 test: 0.277
RMSE - test: 4668.0
MAE - test: 1372.0
MAPE - test: 0.94


c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\linear_model\_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [158]:
# Observar Coeficientes de cada variable para cada modelo en un dataframe
coefficients_huber = pd.DataFrame({'Variable':df_nonull.drop(["Immigrant count"], axis=1, inplace=False).columns})
coefficients_huber['modelo_lineal']= model_huber.coef_

# Mostrar coeficientes
coefficients_huber.head(20)

,Variable,modelo_lineal
0,Year,39.867744
1,Unemployment %,298.585706
2,Political and Violence Percentile,-119.777828
3,Probability of dying young,441.239407
4,Rule of Law Percentile,-141.951313
5,Salaried workers %,-495.752277
6,GDP_growth,-350.461215
7,Inflation_annual,520.854837
8,Liberal democracy index,304.233007
9,Health equality,-350.545236


#### Regresion lineal - RANSAC (ventaja: bueno para grandes outliers en "y")

In [7]:
model_ransac = RANSACRegressor(min_samples = 15) # definicion del modelo 

model_ransac.fit(X_train, y_train) # ajuste del modelo 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_ransac = model_ransac.predict(X_train)
y_test_pred_ransac = model_ransac.predict(X_test)

# Calcular métricas en conjunto train
r2_train_ransac = np.round(r2_score(y_train, y_train_pred_ransac), 3)
rmse_train_ransac = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_ransac)), 0)
mae_train_ransac = np.round(mean_absolute_error(y_train, y_train_pred_ransac), 0)
mape_train_ransac = np.round(mean_absolute_percentage_error(y_train, y_train_pred_ransac), 2)

# Calcular métricas en conjunto test
r2_test_ransac = np.round(r2_score(y_test, y_test_pred_ransac), 3)
rmse_test_ransac = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_ransac)), 0)
mae_test_ransac = np.round(mean_absolute_error(y_test, y_test_pred_ransac), 0)
mape_test_ransac = np.round(mean_absolute_percentage_error(y_test, y_test_pred_ransac), 2)

# Mostrar métricas
print("R2 train:", r2_train_ransac)
print("RMSE - train:", rmse_train_ransac)
print("MAE - train:", mae_train_ransac)
print("MAPE - train:", mape_train_ransac)
print("")
print("R2 test:", r2_test_ransac)
print("RMSE - test:", rmse_test_ransac)
print("MAE - test:", mae_test_ransac)
print("MAPE - test:", mape_test_ransac)

R2 train: -0.031
RMSE - train: 5626.0
MAE - train: 1769.0
MAPE - train: 0.89

R2 test: 0.01
RMSE - test: 5463.0
MAE - test: 1831.0
MAPE - test: 0.69


#### Regresion lineal - TheilSen (ventaja: bueno para outliers pequeños tanto en "X" como en "y")

In [8]:
model_theilsen = TheilSenRegressor() # definicion del modelo 

model_theilsen.fit(X_train, y_train) # ajuste del modelo 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_theilsen = model_theilsen.predict(X_train)
y_test_pred_theilsen = model_theilsen.predict(X_test)

# Calcular métricas en conjunto train
r2_train_theilsen = np.round(r2_score(y_train, y_train_pred_theilsen), 3)
rmse_train_theilsen = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_theilsen)), 0)
mae_train_theilsen = np.round(mean_absolute_error(y_train, y_train_pred_theilsen), 0)
mape_train_theilsen = np.round(mean_absolute_percentage_error(y_train, y_train_pred_theilsen), 2)

# Calcular métricas en conjunto test
r2_test_theilsen = np.round(r2_score(y_test, y_test_pred_theilsen), 3)
rmse_test_theilsen = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_theilsen)), 0)
mae_test_theilsen = np.round(mean_absolute_error(y_test, y_test_pred_theilsen), 0)
mape_test_theilsen = np.round(mean_absolute_percentage_error(y_test, y_test_pred_theilsen), 2)

# Mostrar métricas
print("R2 train:", r2_train_theilsen)
print("RMSE - train:", rmse_train_theilsen)
print("MAE - train:", mae_train_theilsen)
print("MAPE - train:", mape_train_theilsen)
print("")
print("R2 test:", r2_test_theilsen)
print("RMSE - test:", rmse_test_theilsen)
print("MAE - test:", mae_test_theilsen)
print("MAPE - test:", mape_test_theilsen)

R2 train: 0.333
RMSE - train: 4525.0
MAE - train: 1638.0
MAPE - train: 4.78

R2 test: 0.453
RMSE - test: 4061.0
MAE - test: 1600.0
MAPE - test: 3.53


#### Modelos lineales regularizados (Ridge, Lasso, E-Net)

**Buscra ALfa Optimo**

In [9]:
# Definir modelo Ridge y para Evaluar el valor del "alpha" óptimo
ridgecv = RidgeCV()
ridgecv.fit(X_train, y_train)
print("Alfa Optimo Ridge:", ridgecv.alpha_)

# Definir modelo Lasso y para Evaluar el valor del "alpha" óptimo
lassocv = LassoCV()
lassocv.fit(X_train, y_train)
print("Alfa Optimo Lasso:", lassocv.alpha_)

# Definir modelo E-Net y para Evaluar el valor del "alpha" óptimo
enetcv = ElasticNetCV()
enetcv.fit(X_train, y_train)
print("Alfa Optimo E-Net:", enetcv.alpha_)

Alfa Optimo Ridge: 1.0
Alfa Optimo Lasso: 0.92773669106235
Alfa Optimo E-Net: 1.7304206862222227


In [10]:
# Ingresamos el valor de alpha en una variable
alpha_opt_ridge = ridgecv.alpha_

# Ingresamos el valor de alpha en una variable
alpha_opt_lasso = lassocv.alpha_

# Ingresamos el valor de alpha en una variable
alpha_opt_enet = enetcv.alpha_

**Entrenar modelo con Alfa optimo**

In [11]:
# Definir modelo Ridge con nuestro valor optimo de alpha, entrenar y predecir
modelo_ridge = Ridge(alpha = alpha_opt_ridge)
y_test_ridge = modelo_ridge.fit(X_train, y_train).predict(X_test)

# Definir modelo Lasso con nuestro valor optimo de alpha, entrenar y predecir
modelo_lasso = Lasso(alpha = alpha_opt_lasso)
y_test_lasso = modelo_lasso.fit(X_train, y_train).predict(X_test)

# Definir modelo E-Net con nuestro valor optimo de alpha, entrenar y predecir
modelo_enet = ElasticNet(alpha = alpha_opt_enet)
y_test_enet = modelo_enet.fit(X_train, y_train).predict(X_test)

In [12]:
# Observar Coeficientes de cada variable para cada modelo en un dataframe
coefficients = pd.DataFrame({'Variable':df_nonull.drop(["Immigrant count"], axis=1, inplace=False).columns})
coefficients['modelo_ridge']= modelo_ridge.coef_
coefficients['modelo_lasso']= modelo_lasso.coef_
coefficients['modelo_net']= modelo_enet.coef_

# Mostrar coeficientes
coefficients

,Variable,modelo_ridge,modelo_lasso,modelo_net
0,Year,-3147.756559,-3099.376147,211.210996
1,Unemployment %,3380.453482,2904.497608,60.050911
2,Political and Violence Percentile,2538.179203,2426.272784,-56.472574
3,Probability of dying young,6928.625842,6581.308239,40.462542
4,Rule of Law Percentile,-9096.036039,-8292.332375,10.561849
...,...,...,...,...
64,Sub-region_Central America and Caribbean,89.346574,0.000000,-125.055870
65,Sub-region_European Union,366.765265,221.596086,16.642410
66,Sub-region_North America,761.530654,44.891180,-47.132018
67,Sub-region_Rest of Europe,-59.658470,-16.608889,-9.359894


**Evaluar y Comparar Métricas**

In [13]:
# Métricas en test - Ridge
r2_test_ridge = np.round(r2_score(y_test, y_test_ridge), 3)
rmse_test_ridge = np.round(np.sqrt(mean_squared_error(y_test, y_test_ridge)), 0)
mae_test_ridge = np.round(mean_absolute_error(y_test, y_test_ridge), 0)
mape_test_ridge = np.round(mean_absolute_percentage_error(y_test, y_test_ridge), 2)

# Mostrar métricas - Ridge
print("R2 test - Ridge:", r2_test_ridge)
print("RMSE test - Ridge:", rmse_test_ridge)
print("MAE test - Ridge:", mae_test_ridge)
print("MAPE test - Ridge:", mape_test_ridge)

R2 test - Ridge: 0.507
RMSE test - Ridge: 3855.0
MAE test - Ridge: 1800.0
MAPE test - Ridge: 4.56


In [14]:
# Métricas en test - Lasso
r2_test_lasso = np.round(r2_score(y_test, y_test_lasso), 3)
rmse_test_lasso = np.round(np.sqrt(mean_squared_error(y_test, y_test_lasso)), 0)
mae_test_lasso = np.round(mean_absolute_error(y_test, y_test_lasso), 0)
mape_test_lasso = np.round(mean_absolute_percentage_error(y_test, y_test_lasso), 2)

# Mostrar métricas - Lasso
print("R2 test - Lasso:", r2_test_lasso)
print("RMSE test - Lasso:", rmse_test_lasso)
print("MAE test - Lasso:", mae_test_lasso)
print("MAPE test - Lasso:", mape_test_lasso)

R2 test - Lasso: 0.506
RMSE test - Lasso: 3859.0
MAE test - Lasso: 1797.0
MAPE test - Lasso: 4.52


In [15]:
# Métricas en test - E-Net
r2_test_enet = np.round(r2_score(y_test, y_test_enet), 3)
rmse_test_enet = np.round(np.sqrt(mean_squared_error(y_test, y_test_enet)), 0)
mae_test_enet = np.round(mean_absolute_error(y_test, y_test_enet), 0)
mape_test_enet = np.round(mean_absolute_percentage_error(y_test, y_test_enet), 2)

# Mostrar métricas - E-Net
print("R2 test - E-Net:", r2_test_enet)
print("RMSE test - E-Net:", rmse_test_enet)
print("MAE test - E-Net:", mae_test_enet)
print("MAPE test - E-Net:", mape_test_enet)

R2 test - E-Net: 0.103
RMSE test - E-Net: 5199.0
MAE test - E-Net: 2293.0
MAPE test - E-Net: 6.1


#### Desicion Tree

In [168]:
# Definir diccionario de valores para parámetros 
params = {'max_depth': range(6,8), 
          'min_samples_leaf' : [1, 3, 4, 5], 
          'min_samples_split': [20, 30], 
          "criterion" : ["squared_error", "absolute_error", "poisson"] 
          } 

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
tree = DecisionTreeRegressor() 
tree_cv = GridSearchCV(tree, params, cv = 3, refit = True, scoring = "neg_mean_squared_error")

# Entrenar modelo con cada combinación de parámetro 
tree_cv.fit(X_train, y_train) 

# Montrar los valores de los parámetros 
print(tree_cv.best_params_)

{'criterion': 'squared_error', 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 20}


In [17]:
# Definir modelo con los mejores valores de parámetros 
tree_best =  DecisionTreeRegressor(max_depth = 7, 
                                   min_samples_leaf = 2,
                                   min_samples_split = 10, 
                                   criterion = tree_cv.best_params_['criterion']) 

# Entrenar con el conjunto de entrenamiento 
tree_best.fit(X_train, y_train) 

 
# Aplicar modelo sobre los datos de traint y test para predecir el target
y_test_pred_tree = tree_best.predict(X_test) 
y_train_pred_tree = tree_best.predict(X_train) 

# Calcular métricas en conjunto train
r2_train_tree = np.round(r2_score(y_train, y_train_pred_tree), 3)
rmse_train_tree = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_tree)), 0)
mae_train_tree = np.round(mean_absolute_error(y_train, y_train_pred_tree), 0)
mape_train_tree = np.round(mean_absolute_percentage_error(y_train, y_train_pred_tree), 2)

# Calcular métricas en conjunto test
r2_test_tree = np.round(r2_score(y_test, y_test_pred_tree), 3)
rmse_test_tree = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_tree)), 0)
mae_test_tree = np.round(mean_absolute_error(y_test, y_test_pred_tree), 0)
mape_test_tree = np.round(mean_absolute_percentage_error(y_test, y_test_pred_tree), 2)

# Mostrar métricas
print("R2 - train:", r2_train_tree)
print("RMSE - train:", rmse_train_tree)
print("MAE - train:", mae_train_tree)
print("MAPE - train:", mape_train_tree)
print("")
print("R2 - test:", r2_test_tree)
print("RMSE - test:", rmse_test_tree)
print("MAE - test:", mae_test_tree)
print("MAPE - test:", mape_test_tree)


R2 - train: 0.823
RMSE - train: 2328.0
MAE - train: 990.0
MAPE - train: 2.14

R2 - test: 0.662
RMSE - test: 3194.0
MAE - test: 1216.0
MAPE - test: 1.67


#### Random Forest

In [177]:
# Definir diccionario de valores para parámetros 
params = {'n_estimators': [100], 
	      'criterion' : ['squared_error', 'friedman_mse', 'poisson'],
          "min_samples_split": [30, 50, 70], 
          'min_samples_leaf' : [2, 3, 5],
          "max_depth": [7, 8]
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
rf = RandomForestRegressor() 
rf_cv = GridSearchCV(rf, params, cv=3, scoring='neg_mean_squared_error').fit(X_train, y_train)

# Motrar mejores valores para parámeros
rf_cv.best_estimator_

RandomForestRegressor(criterion='poisson', max_depth=8, min_samples_leaf=2,
                      min_samples_split=30)

In [18]:
# Definir modelo con los mejores valores de parámetros (Nota: usar los mejores, pero hacer modificaciones para comparar metricas)
rf_best = RandomForestRegressor(n_estimators = 200, 
                           max_depth = 8, 
                           criterion = 'poisson', 
                           min_samples_split = 15,
                           min_samples_leaf = 2,
                           )

# Entrenar con el conjunto de entrenamiento 
rf_best.fit(X_train, y_train) 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_rf = rf_best.predict(X_train)
y_test_pred_rf = rf_best.predict(X_test)

# Calcular métricas en conjunto train
r2_train_rf = np.round(r2_score(y_train, y_train_pred_rf), 3)
rmse_train_rf = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_rf)), 0)
mae_train_rf = np.round(mean_absolute_error(y_train, y_train_pred_rf), 0)
mape_train_rf = np.round(mean_absolute_percentage_error(y_train, y_train_pred_rf), 2)

# Calcular métricas en conjunto test
r2_test_rf = np.round(r2_score(y_test, y_test_pred_rf), 3)
rmse_test_rf = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_rf)), 0)
mae_test_rf = np.round(mean_absolute_error(y_test, y_test_pred_rf), 0)
mape_test_rf = np.round(mean_absolute_percentage_error(y_test, y_test_pred_rf), 2)

# Mostrar métricas
print("R2 - train:", r2_train_rf)
print("RMSE - train:", rmse_train_rf)
print("MAE - train:", mae_train_rf)
print("MAPE - train:", mape_train_rf)
print("")
print("R2 - test:", r2_test_rf)
print("RMSE - test:", rmse_test_rf)
print("MAE - test:", mae_test_rf)
print("MAPE - test:", mape_test_rf)

R2 - train: 0.833
RMSE - train: 2266.0
MAE - train: 718.0
MAPE - train: 0.81

R2 - test: 0.773
RMSE - test: 2614.0
MAE - test: 884.0
MAPE - test: 0.78


In [19]:
# Estimación de importancia relativa de variables en el modelo
imp_rel_rf = rf_best.feature_importances_
importancias_rf = pd.DataFrame({"variable": X.columns, "importancia relativa": imp_rel_rf}) \
.sort_values(by='importancia relativa', ascending = False)
importancias_rf[:15]

,variable,importancia relativa
57,Age group_All,0.464272
15,Number of residents,0.264842
0,Year,0.034406
52,Age group_25 - 34,0.027380
18,Number of Turist,0.023383
1,Unemployment %,0.020307
56,Age group_65+,0.018371
21,Año post_pandemia,0.017214
3,Probability of dying young,0.013992
55,Age group_55 - 64,0.013036


#### KNN

In [186]:
# Definir diccionario de valores para parámetros 
params = {'n_neighbors': range(1,20),
          'weights' : ['uniform', 'distance'],
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario entrenando el conjunto train
knn = KNeighborsRegressor()
knn_cv = GridSearchCV(knn, params, cv=3, scoring='neg_mean_squared_error').fit(X_train,y_train)

knn_cv.best_params_

{'n_neighbors': 1, 'weights': 'uniform'}

In [20]:
knn_best =  KNeighborsRegressor(n_neighbors = 5, leaf_size=30, p = 1)

knn_best.fit(X_train, y_train)

# Obtener predicciones con conjunto de entrenamiento y prueba
y_train_pred_knn = knn_best.predict(X_train)  
y_test_pred_knn = knn_best.predict(X_test)  

# Calcular métricas en conjunto train
r2_train_knn = np.round(r2_score(y_train, y_train_pred_knn), 3)
rmse_train_knn = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_knn)), 0)
mae_train_knn = np.round(mean_absolute_error(y_train, y_train_pred_knn), 0)
mape_train_knn = np.round(mean_absolute_percentage_error(y_train, y_train_pred_knn), 2)

# Calcular métricas en conjunto train
r2_test_knn = np.round(r2_score(y_test, y_test_pred_knn), 3)
rmse_test_knn = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_knn)), 0)
mae_test_knn = np.round(mean_absolute_error(y_test, y_test_pred_knn), 0)
mape_test_knn = np.round(mean_absolute_percentage_error(y_test, y_test_pred_knn), 2)

# Print metrics
print("R2 - train:", r2_train_knn)
print("RMSE - train:", rmse_train_knn)
print("MAE - train:", mae_train_knn)
print("MAPE - train:", mape_train_knn)
print("")
print("R2 - test:", r2_test_knn)
print("RMSE - test:", rmse_test_knn)
print("MAE - test:", mae_test_knn)
print("MAPE - test:", mape_test_knn)

R2 - train: 0.685
RMSE - train: 3111.0
MAE - train: 873.0
MAPE - train: 0.56

R2 - test: 0.46
RMSE - test: 4035.0
MAE - test: 1193.0
MAPE - test: 0.8


#### SVR

In [189]:
# Definir diccionario de valores para parámetros 
params = {'kernel': ['rbf', 'linear'],
          'gamma': ['scale', 'auto'],
          'C' : [1.0, 0.85, 0.75] ,
          'max_iter': [-1, 100],
          "tol" : [0.001, 0.002, 0.0015, 0.1, 0.2]
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
svr = SVR()
svr_cv = GridSearchCV(svr, params, cv = 3, refit = True, scoring = 'neg_mean_squared_error') # elegir scoring deseano (r2, mae, mse, mape...)

# Entrenar modelo con cada combinación de parámetro 
svr_cv.fit(X_train, y_train)

# Motrar mejores valores para parámeros
svr_cv.best_params_

c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=

{'C': 1.0, 'gamma': 'scale', 'kernel': 'linear', 'max_iter': -1, 'tol': 0.2}

In [21]:
# Definir modelo con mejors parámetros
svr_best = SVR(kernel = 'linear', 
               gamma = 'scale', 
               C = 1.0, 
               max_iter = -1, 
               tol = 0.2
               )

# Enrenar con el conjunto de entrenamiento 
svr_best.fit(X_train, y_train) 

# Obtener predicciones con conjunto de entrenamiento y prueba
y_train_pred_svr = svr_best.predict(X_train)  
y_test_pred_svr = svr_best.predict(X_test)  

# Calcular métricas en conjunto train
r2_train_svr = np.round(r2_score(y_train, y_train_pred_svr), 3)
rmse_train_svr = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_svr)), 0)
mae_train_svr = np.round(mean_absolute_error(y_train, y_train_pred_svr), 0)
mape_train_svr = np.round(mean_absolute_percentage_error(y_train, y_train_pred_svr), 2)

# Calcular métricas en conjunto test
r2_test_svr = np.round(r2_score(y_test, y_test_pred_svr), 3)
rmse_test_svr = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_svr)), 0)
mae_test_svr = np.round(mean_absolute_error(y_test, y_test_pred_svr), 0)
mape_test_svr = np.round(mean_absolute_percentage_error(y_test, y_test_pred_svr), 2)

# Mostrar métricas
print("R2 - train:", r2_train_svr)
print("RMSE - train:", rmse_train_svr)
print("MAE - train:", mae_train_svr)
print("MAPE - train:", mape_train_svr)
print("")
print("R2 - test:", r2_test_svr)
print("RMSE - test:", rmse_test_svr)
print("MAE - test:", mae_test_svr)
print("MAPE - test:", mape_test_svr)


R2 - train: 0.026
RMSE - train: 5469.0
MAE - train: 1659.0
MAPE - train: 1.04

R2 - test: 0.022
RMSE - test: 5430.0
MAE - test: 1758.0
MAPE - test: 1.01


#### Red Neuronal Básica

Obtengamos primero un número de referencia en cuanto a el número de neuronas a usar. 

In [22]:
# Dos métodos para estimar numero de neuronas a usar
print(np.sqrt(len(X.columns)*2))
print(2/3 * len(X.columns) + 2)

11.74734012447073
48.0


In [193]:
# Definir diccionario de valores para parámetros 
params = {'max_iter': [200],
          'hidden_layer_sizes':[11, 44, (44, 11)],
          'batch_size': [30, 50],
          'activation': ['relu', 'identity', 'tanh'],
          'alpha': [0.1, 0.01],
          'early_stopping' : [True],
          'solver' : ['adam', 'sgd', 'lbfgs']}

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
rn = MLPRegressor()
rn_cv = GridSearchCV(rn, param_grid = params, cv = 3, scoring='neg_mean_squared_error', verbose = True, n_jobs = -1)

# Entrenar modelo con cada combinación de parámetro 
rn_cv.fit(X_train, y_train)

# Motrar mejores valores para parámeros
rn_cv.best_params_

Fitting 3 folds for each of 108 candidates, totalling 324 fits


c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
36 fits failed out of a total of 324.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 749, in fit
    return self._fit(X, y, incremental=False)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptr

{'activation': 'relu',
 'alpha': 0.1,
 'batch_size': 30,
 'early_stopping': True,
 'hidden_layer_sizes': 44,
 'max_iter': 200,
 'solver': 'lbfgs'}

In [28]:
# hacer un RN con los mejores parametros obtenidos y entrenar
rn_best = MLPRegressor(activation = 'relu',
                         alpha = 0.01, 
                         batch_size= 30, 
                         early_stopping = True, 
                         hidden_layer_sizes = 44, 
                         max_iter = 200, 
                         solver = 'lbfgs',
                         )

# Entrenar con el conjunto de entrenamiento 
rn_best.fit(X_train, y_train)

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_test_pred_rn = rn_best.predict(X_test) 
y_train_pred_rn = rn_best.predict(X_train) 

# Calculo de metricas en train
r2_train_rn = np.round(r2_score(y_train, y_train_pred_rn), 3)
rmse_train_rn = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_rn)), 0)
mae_train_rn = np.round(mean_absolute_error(y_train, y_train_pred_rn), 0)
mape_train_rn = np.round(mean_absolute_percentage_error(y_train, y_train_pred_rn), 2)

# Calculo de metricas en test
r2_test_rn = np.round(r2_score(y_test, y_test_pred_rn), 3)
# rmse_test_rn = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_rn)), 0) 
mae_test_rn = np.round(mean_absolute_error(y_test, y_test_pred_rn), 0)
# mape_test_rn = np.round(mean_absolute_percentage_error(y_test, y_test_pred_rn), 2) 

# Mostrar métricas
print("R2 - train:", r2_train_rn)
print("RMSE - train:", rmse_train_rn)
print("MAE - train:", mae_train_rn)
print("MAPE - train:", mape_train_rn)
print("")
print("R2 - test:", r2_test_rn)
# print("RMSE - test:", rmse_test_rn)  
print("MAE - test:", mae_test_rn)
# print("MAPE - test:", mape_test_rn) 

R2 - train: 0.976
RMSE - train: 866.0
MAE - train: 522.0
MAPE - train: 0.78

R2 - test: 0.94
MAE - test: 629.0


c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


### *Modelos Que Aceptan Datos Nulos*

Ahora evalueremos los datos con dos modelos que aceptan datos nulos (Hist Gradient Boosting y XGBoost), por lo que haremos una copia del conunto de datos con todos los países y, al igual que antes, haremos de la variable "Year" una variable ordinal y el resto de variable categóricas a variables *dummy*.

In [30]:
# hacer copia del df
df_copy = df.copy()

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_copy['Year'] = df_copy['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_copy = pd.get_dummies(df_copy)

# Convertir las variables dummies booleanas en "int"
col_bool = df_copy.select_dtypes(include = ['bool']).columns
df_copy[col_bool] = df_copy[col_bool].astype(int)

# Verificar cambio
df_copy.info()
df_copy

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 71 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year                                      9360 non-null   int64  
 1   Immigrant count                           9360 non-null   int64  
 2   Unemployment %                            9360 non-null   float64
 3   Political and Violence Percentile         9360 non-null   float64
 4   Probability of dying young                9360 non-null   float64
 5   Rule of Law Percentile                    9360 non-null   float64
 6   Salaried workers %                        9360 non-null   float64
 7   GDP_growth                                9360 non-null   float64
 8   Inflation_annual                          9360 non-null   float64
 9   Liberal democracy index                   9360 non-null   float64
 10  Health equality                     

,Year,Immigrant count,Unemployment %,Political and Violence Percentile,Probability of dying young,Rule of Law Percentile,Salaried workers %,GDP_growth,Inflation_annual,Liberal democracy index,...,Continent_America,Continent_Asia,Continent_Europe,Sub-region_Africa,Sub-region_Asia,Sub-region_Central America and Caribbean,Sub-region_European Union,Sub-region_North America,Sub-region_Rest of Europe,Sub-region_South America
0,1,759,11.33,14.90,3.7,24.52,67.41,2.40,15.31,0.164,...,0,0,0,1,0,0,0,0,0,0
1,1,2938,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
2,1,1128,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
3,1,265,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
4,1,156,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,15,330,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9356,15,146,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9357,15,169,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9358,15,99,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0


Separemos el conjunto train/test y escalemos los datos.

In [31]:
# Separar variables input y variable target "Immigrant count" de df_copy
X = df_copy.drop("Immigrant count", axis = 1) # variables predictoras
y = df_copy["Immigrant count"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 58) # separar datos en conjunto train y test en un 75% / 25%
scaler = MinMaxScaler() # definir scaler de datos 
X_train = scaler.fit_transform(X_train) # escalar los datos de entrenamiento
X_test = scaler.fit_transform(X_test) # # escalar los datos de prueba

#### Hist Gradient Boosting

In [208]:
# Definir diccionario de valores para parámetros 
params = {'max_iter': [120], 
	      'loss' : ['squared_error', 'gamma', 'poisson'],
          "learning_rate": [0.1, 0.01], 
          'min_samples_leaf' : [2, 3, 5],
          "max_depth": [7, 8],
          'l2_regularization' : [0.0, 0.1, 0.3] #usar si se tienen muchas variables
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
hgb = HistGradientBoostingRegressor() 
hgb_cv = GridSearchCV(hgb, params, cv=3, scoring='neg_mean_squared_error').fit(X_train, y_train)

hgb_cv.best_estimator_

c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
108 fits failed out of a total of 324.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\ensemble\_hist_gradient_boosting\gradient_boosting.py", line 353, in fit
    self._validate_params()
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "

HistGradientBoostingRegressor(l2_regularization=0.3, max_depth=8, max_iter=120,
                              min_samples_leaf=5)

In [32]:
# Definir modelo con los mejores valores de parámetros (Nota: usar los mejores, pero hacer modificaciones para comparar metricas)
hgb_best = HistGradientBoostingRegressor(
                           max_iter = 120, 
                           max_depth = 8,
                           loss = 'poisson',
                           learning_rate = 0.1, 
                           min_samples_leaf = 35,
                           max_leaf_nodes = 25,
                           )

# Entrenar con el conjunto de entrenamiento 
hgb_best.fit(X_train, y_train) 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_hgb = hgb_best.predict(X_train)
y_test_pred_hgb = hgb_best.predict(X_test)

# Calculo de metricas en train
r2_train_hgb = np.round(r2_score(y_train, y_train_pred_hgb), 3)
rmse_train_hgb = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_hgb)), 0)
mae_train_hgb = np.round(mean_absolute_error(y_train, y_train_pred_hgb), 0)
mape_train_hgb = np.round(mean_absolute_percentage_error(y_train, y_train_pred_hgb), 2)

# Calculo de metricas en train
r2_test_hgb = np.round(r2_score(y_test, y_test_pred_hgb), 3)
rmse_test_hgb = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_hgb)), 0)
mae_test_hgb = np.round(mean_absolute_error(y_test, y_test_pred_hgb), 0)
mape_test_hgb = np.round(mean_absolute_percentage_error(y_test, y_test_pred_hgb), 2)

# Mostrar métricas
print("R2 - train:", r2_train_hgb)
print("RMSE - train:", rmse_train_hgb)
print("MAE - train:", mae_train_hgb)
print("MAPE - train:", mape_train_hgb)
print('')
print("R2 - test:", r2_test_hgb)
print("RMSE - test:", rmse_test_hgb)
print("MAE - test:", mae_test_hgb)
print("MAPE - test:", mape_test_hgb)

R2 - train: 0.988
RMSE - train: 589.0
MAE - train: 275.0
MAPE - train: 0.33

R2 - test: 0.94
RMSE - test: 1342.0
MAE - test: 417.0
MAPE - test: 0.39


In [217]:
# Importancia por permutaciones
importancias_permu = permutation_importance(hgb_best, X_train, y_train, n_repeats=10, random_state=58)

# Importancias en dataframe
importances_hgb = pd.DataFrame({
    'feature': X.columns,
    'importance_mean': importancias_permu.importances_mean,
    'importance_std': importancias_permu.importances_std
}).sort_values(by='importance_mean', ascending=False)

importances_hgb.head(15)

,feature,importance_mean,importance_std
58,Age group_All,1.053554,0.065777
15,Number of residents,0.646439,0.078739
0,Year,0.226384,0.028672
18,Number of Turist,0.125457,0.017874
1,Unemployment %,0.101551,0.013185
48,Sex_Both,0.095145,0.021593
57,Age group_65+,0.064284,0.007790
5,Salaried workers %,0.051695,0.008413
56,Age group_55 - 64,0.045938,0.010805
7,Inflation_annual,0.038755,0.002671


*NOTA: USAR QUANTILES EN EL HGB PARA ESTIMAR INTERVALO DE CONFIANZA.*

#### XGBoost

In [218]:
# Definir diccionario de valores para parámetros 
params = {'objective' : ['reg:squarederror'], # seleccionar 2 como max, reg:squarederror es el default
          'eval_metric' : ['rmse'],   # rmsle disminuye efecto de outliers
          'booster' : ["gbtree", "gblinear", "dart"], # gbtree (default) y dart estan basados en arboles
          'n_estimators': [200],
          'max_depth': [7, 9], # dependiendo de la dimensionalidad de los datos, usar valores de profundidad menor
          "learning_rate" : [0.1, 0.05], 
       #   "colsample_bytree" : [1, 0.7], # porcentaje de variables a usar (buen parametro para cuando se tiene una gran cantidad de variables)
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
xgb = XGBRegressor()
xgb_cv = GridSearchCV(xgb, params, cv=3, scoring = 'neg_mean_squared_error') # elegir scoring deseano (r2, mae, mse, mape...)

# Entrenar modelo con cada combinación de parámetro 
xgb_cv.fit(X_train,y_train)

# Motrar mejores valores para parámeros
xgb_cv.best_params_ 

C:\Users\Cristian De Andrade\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [22:46:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Cristian De Andrade\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [22:46:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Cristian De Andrade\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [22:46:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  w

{'booster': 'gbtree',
 'eval_metric': 'rmse',
 'learning_rate': 0.1,
 'max_depth': 7,
 'n_estimators': 200,
 'objective': 'reg:squarederror'}

In [33]:
# Definir modelo con los mejores valores de parámetros (Nota: usar los mejores, pero hacer modificaciones al comparar metricas)
xgb_best = XGBRegressor(objective = 'reg:squarederror', 
                        eval_metric = 'rmse',
                        booster = 'gbtree',
                        n_estimators = 200,
                        max_depth = 8,
                        learning_rate = 0.13,
                        min_child_weight = 25, 
                    )

# Entrenar modelo con el conjunto de entrenamiento 
xgb_best.fit(X_train, y_train)

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_xgb = xgb_best.predict(X_train) 
y_test_pred_xgb = xgb_best.predict(X_test) 

# Calculate metrics for train set
r2_train_xgb = np.round(r2_score(y_train, y_train_pred_xgb), 3)
rmse_train_xgb = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_xgb)), 0)
mae_train_xgb = np.round(mean_absolute_error(y_train, y_train_pred_xgb), 0)
mape_train_xgb = np.round(mean_absolute_percentage_error(y_train, y_train_pred_xgb), 2)

# Calculate metrics for test set
r2_test_xgb = np.round(r2_score(y_test, y_test_pred_xgb), 3)
rmse_test_xgb = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_xgb)), 0)
mae_test_xgb = np.round(mean_absolute_error(y_test, y_test_pred_xgb), 0)
mape_test_xgb = np.round(mean_absolute_percentage_error(y_test, y_test_pred_xgb), 2)

# Print metrics
print("R2 - train:", r2_train_xgb)
print("RMSE - train:", rmse_train_xgb)
print("MAE - train:", mae_train_xgb)
print("MAPE - train:", mape_train_xgb)
print("")
print("R2 - test:", r2_test_xgb)
print("RMSE - test:", rmse_test_xgb)
print("MAE - test:", mae_test_xgb)
print("MAPE - test:", mape_test_xgb)


R2 - train: 0.968
RMSE - train: 967.0
MAE - train: 338.0
MAPE - train: 0.69

R2 - test: 0.877
RMSE - test: 1923.0
MAE - test: 565.0
MAPE - test: 0.88


### *Comparar Modelos*

Juntemos los resultados todos los modelos en un dataframe por tipo de métrica, ordenando por el mejor valor en el conjunto test.

In [34]:
# Juntar en un dataframe los datos
modelos_r2 = pd.DataFrame({
    'Modelo' : ['Lineal', 'Lineal - Huber', 'Lineal - RANSAC', 'Lineal - Theilsen', 'Lineal - Ridge', 'Lineal - Lasso', 'Lineal - E-Net', 'Decision Tree', 'Random Forest', 'KNN', 'SVR', 'Red Neuronal', 'HGB', 'XGBoost'],
    'R² train' : [r2_train_lineal, r2_train_huber, r2_train_ransac, r2_train_theilsen, np.nan, np.nan, np.nan, r2_train_tree, r2_train_rf, r2_train_knn, r2_train_svr, r2_train_rn, r2_train_hgb, r2_train_xgb],
    'R² test' : [r2_test_lineal, r2_test_huber, r2_test_ransac, r2_test_theilsen, r2_test_ridge, r2_test_lasso, r2_test_enet, r2_test_tree, r2_test_rf, r2_test_knn, r2_test_svr, r2_test_rn, r2_test_hgb, r2_test_xgb],
})

# Ordenar de forma descendente por R² test
modelos_r2.sort_values(by = 'R² test', ascending = False)

,Modelo,R² train,R² test
11,Red Neuronal,0.976,0.940
12,HGB,0.988,0.940
13,XGBoost,0.968,0.877
8,Random Forest,0.833,0.773
7,Decision Tree,0.823,0.662
0,Lineal,0.445,0.507
4,Lineal - Ridge,NaN,0.507
5,Lineal - Lasso,NaN,0.506
9,KNN,0.685,0.460
3,Lineal - Theilsen,0.333,0.453


In [35]:
# Juntar en un dataframe los datos
modelos_rsme = pd.DataFrame({
    'Modelo' : ['Lineal', 'Lineal - Huber', 'Lineal - RANSAC', 'Lineal - Theilsen', 'Lineal - Ridge', 'Lineal - Lasso', 'Lineal - E-Net', 'Decision Tree', 'Random Forest', 'KNN','SVR', 'Red Neuronal', 'HGB', 'XGBoost'],
    'RSME train' : [rmse_train_lineal, rmse_train_huber, rmse_train_ransac, rmse_train_theilsen, np.nan, np.nan, np.nan, rmse_train_tree, rmse_train_rf, rmse_train_knn, rmse_train_svr, rmse_train_rn, rmse_train_hgb, rmse_train_xgb],
    'RSME test' : [rmse_test_lineal, rmse_test_huber, rmse_test_ransac, rmse_test_theilsen, rmse_test_ridge, rmse_test_lasso, rmse_test_enet, rmse_test_tree, rmse_test_rf, rmse_test_knn, rmse_test_svr, np.nan, rmse_test_hgb, rmse_test_xgb],
    })

# Ordenar de forma ascendente por RSME test
modelos_rsme.sort_values(by = 'RSME test', ascending = True)

,Modelo,RSME train,RSME test
12,HGB,589.0,1342.0
13,XGBoost,967.0,1923.0
8,Random Forest,2266.0,2614.0
7,Decision Tree,2328.0,3194.0
0,Lineal,4128.0,3853.0
4,Lineal - Ridge,NaN,3855.0
5,Lineal - Lasso,NaN,3859.0
9,KNN,3111.0,4035.0
3,Lineal - Theilsen,4525.0,4061.0
1,Lineal - Huber,4800.0,4668.0


In [39]:
# Juntar en un dataframe los datos
modelos_mae = pd.DataFrame({
    'Modelo' : ['Lineal', 'Lineal - Huber', 'Lineal - RANSAC', 'Lineal - Theilsen', 'Lineal - Ridge', 'Lineal - Lasso', 'Lineal - E-Net', 'Decision Tree', 'Random Forest', 'KNN', 'SVR', 'Red Neuronal', 'HGB', 'XGBoost'],
    'MAE train' : [mae_train_lineal, mae_train_huber, mae_train_ransac, mae_train_theilsen, np.nan, np.nan, np.nan, mae_train_tree, mae_train_rf, mae_train_knn, mae_train_svr, mae_train_rn, mae_train_hgb, mae_train_xgb],
    'MAE test' : [mae_test_lineal, mae_test_huber, mae_test_ransac, mae_test_theilsen, mae_test_ridge, mae_test_lasso, mae_test_enet, mae_test_tree, mae_test_rf, mae_test_knn, mae_test_svr, mae_test_rn, mae_test_hgb, mae_test_xgb],
})

# Ordenar de forma descendente por MAE test
modelos_mae.sort_values(by = 'MAE test', ascending = True)

,Modelo,MAE train,MAE test
12,HGB,275.0,417.0
13,XGBoost,338.0,565.0
11,Red Neuronal,522.0,629.0
8,Random Forest,718.0,884.0
9,KNN,873.0,1193.0
7,Decision Tree,990.0,1216.0
1,Lineal - Huber,1306.0,1372.0
3,Lineal - Theilsen,1638.0,1600.0
10,SVR,1659.0,1758.0
5,Lineal - Lasso,NaN,1797.0


In [37]:
# Juntar en un dataframe los datos
modelos_mape = pd.DataFrame({
    'Modelo' : ['Lineal', 'Lineal - Huber', 'Lineal - RANSAC', 'Lineal - Theilsen', 'Lineal - Ridge', 'Lineal - Lasso', 'Lineal - E-Net', 'Decision Tree', 'Random Forest', 'KNN', 'SVR', 'Red Neuronal', 'HGB', 'XGBoost'],
    'MAPE train' : [mape_train_lineal, mape_train_huber, mape_train_ransac, mape_train_theilsen, np.nan, np.nan, np.nan, mape_train_tree, mape_train_rf, mape_train_knn, mape_train_svr, mape_train_rn, mape_train_hgb, mape_train_xgb],
    'MAPE test' : [mape_test_lineal, mape_test_huber, mape_test_ransac, mape_test_theilsen, mape_test_ridge, mape_test_lasso, mape_test_enet, mape_test_tree, mape_test_rf, mape_test_knn, mape_test_svr, np.nan, mape_test_hgb, mape_test_xgb],
})

# Ordenar de forma ascendente por MAPE test
modelos_mape.sort_values(by = 'MAPE test', ascending = True)

,Modelo,MAPE train,MAPE test
12,HGB,0.33,0.39
2,Lineal - RANSAC,0.89,0.69
8,Random Forest,0.81,0.78
9,KNN,0.56,0.80
13,XGBoost,0.69,0.88
1,Lineal - Huber,1.39,0.94
10,SVR,1.04,1.01
7,Decision Tree,2.14,1.67
3,Lineal - Theilsen,4.78,3.53
5,Lineal - Lasso,NaN,4.52


In [43]:
# Exportar tablas comparativas de metricas
modelos_r2.to_csv("../16 - Exports Modelos/metrics_r2.csv", index = False)
modelos_rsme.to_csv("../16 - Exports Modelos/metrics_rsme.csv", index = False)
modelos_mae.to_csv("../16 - Exports Modelos/metrics_mae.csv", index = False)
modelos_mape.to_csv("../16 - Exports Modelos/metrics_mape.csv", index = False)

# Exportar modelo HGB seleccionado
with open("../16 - Exports Modelos/hgb_best.sav", 'wb') as file:
    pickle.dump(hgb_best, file)
